In [1]:
# ===========================
# Collate experiments: teacher (4-epoch) + NEW students only (relative drops)
# Paste & run as single cell in Colab
# ===========================
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!pip install -q transformers datasets evaluate accelerate scikit-learn openpyxl

import os, gc, re, torch, numpy as np, pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
from tqdm.auto import tqdm

# ---------- User paths ----------
DRIVE_BASE = "/content/drive/MyDrive/Colab Notebooks/CodeMix"
test_csv  = os.path.join(DRIVE_BASE, "test.csv")

# NEW: use the 4-epoch teacher folder
teacher_base_dir = os.path.join(DRIVE_BASE, "results_teacher_4epoch", "model")

# Student runs root
STUDENTS_BASE = os.path.join(DRIVE_BASE, "results_students")

# Output files
OUT_CSV  = os.path.join(DRIVE_BASE, "distillation_results_rel_new_only.csv")
OUT_XLSX = os.path.join(DRIVE_BASE, "distillation_results_rel_new_only.xlsx")

CHECKPOINT = "distilbert-base-multilingual-cased"
MAX_LEN = 64
EVAL_BATCH = 32

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------- Load test ----------
if not os.path.exists(test_csv):
    raise FileNotFoundError(f"Test CSV not found at {test_csv}. Run data-prep notebook first.")
test_df = pd.read_csv(test_csv)
if "review" not in test_df.columns or "label" not in test_df.columns:
    raise RuntimeError("Expecting test.csv to have columns 'review' and 'label'.")

test_texts = test_df["review"].astype(str).tolist()
test_labels = test_df["label"].astype(int).tolist()

tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
enc = tokenizer(test_texts, padding="max_length", truncation=True, max_length=MAX_LEN, return_tensors="pt")
indices = list(range(len(test_texts)))
batches = [indices[i:i+EVAL_BATCH] for i in range(0, len(test_texts), EVAL_BATCH)]

# ---------- Metrics ----------
acc_metric = evaluate.load("accuracy")
f1_metric  = evaluate.load("f1")

def evaluate_model(model, device):
    model.to(device).eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch_idx in tqdm(batches, desc="eval batches"):
            logits = model(
                input_ids=enc["input_ids"][batch_idx].to(device),
                attention_mask=enc["attention_mask"][batch_idx].to(device)
            ).logits
            preds = torch.argmax(logits, dim=-1).cpu().numpy().tolist()
            all_preds.extend(preds)
            all_labels.extend([test_labels[i] for i in batch_idx])
    acc = acc_metric.compute(predictions=all_preds, references=all_labels)["accuracy"]
    f1  = f1_metric.compute(predictions=all_preds, references=all_labels, average="macro")["f1"]
    return float(acc), float(f1)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

def extract_layers(model):
    cfg = getattr(model, "config", None)
    if cfg is None: return None
    return getattr(cfg, "num_hidden_layers", None) or getattr(cfg, "n_layers", None) or getattr(cfg, "num_layers", None)

# ---------- Loader (robust to local folder structure) ----------
def load_local_model(folder):
    tried = []
    try:
        return AutoModelForSequenceClassification.from_pretrained(
            folder, local_files_only=True, output_hidden_states=True, output_attentions=True
        )
    except Exception as e:
        tried.append((folder, str(e)))
    for alt in [os.path.join(folder, "student_model"), os.path.join(folder, "model")]:
        if os.path.isdir(alt):
            try:
                return AutoModelForSequenceClassification.from_pretrained(
                    alt, local_files_only=True, output_hidden_states=True, output_attentions=True
                )
            except Exception as e:
                tried.append((alt, str(e)))
    if os.path.isdir(folder):
        ckpts = sorted([os.path.join(folder, d) for d in os.listdir(folder) if d.startswith("checkpoint")])
        for c in reversed(ckpts):
            try:
                return AutoModelForSequenceClassification.from_pretrained(
                    c, local_files_only=True, output_hidden_states=True, output_attentions=True
                )
            except Exception as e:
                tried.append((c, str(e)))
    raise RuntimeError(f"Cannot load model from {folder}. Attempts: {tried}")

# ---------- Helpers ----------
def label_from_name(name):
    n = os.path.basename(name).lower()
    if "baseline" in n: return "Baseline"
    if "soft" in n:     return "Soft"
    if "hidden" in n:   return "Hidden"
    if "full" in n:     return "Full"
    if "embed" in n or "embedding" in n: return "Embedding"
    if "attn" in n or "attention" in n:  return "Attention"
    return os.path.basename(name)

def parse_meta_from_name(name):
    n = os.path.basename(name)
    layers = None; seed = None
    m = re.search(r"layers(\d+)", n)
    if m:  layers = int(m.group(1))
    m2 = re.search(r"seed(\d+)", n)
    if m2: seed = int(m2.group(1))
    return layers, seed

# ---------- Evaluate ----------
records = []

# Teacher (4-epoch)
if not os.path.isdir(teacher_base_dir):
    raise RuntimeError(f"Teacher (4-epoch) not found at {teacher_base_dir}")

print("Loading teacher (4-epoch) from:", teacher_base_dir)
teacher = load_local_model(teacher_base_dir)
teacher_acc, teacher_f1 = evaluate_model(teacher, device)
teacher_params = count_parameters(teacher) / 1e6
teacher_layers = extract_layers(teacher)

records.append({
    "Label": "Teacher",
    "Accuracy": teacher_acc,
    "Macro-F1": teacher_f1,
    "RelAccDrop%": None,
    "RelF1Drop%": None,
    "Layers": teacher_layers,
    "Params(M)": round(teacher_params, 2),
    "ParamReduction%": None,
    "Path": teacher_base_dir,
    "Seed": None
})

teacher.to("cpu"); del teacher; gc.collect(); torch.cuda.empty_cache()

# Students: ONLY the *new* runs tagged with layers# AND teacher4epoch in name
if os.path.isdir(STUDENTS_BASE):
    all_dirs = sorted([os.path.join(STUDENTS_BASE, d) for d in os.listdir(STUDENTS_BASE) if os.path.isdir(os.path.join(STUDENTS_BASE, d))])
else:
    all_dirs = []

new_student_dirs = [
    p for p in all_dirs
    if re.search(r"layers\d+", os.path.basename(p).lower()) and ("teacher4epoch" in os.path.basename(p).lower())
]

if not new_student_dirs:
    print("No NEW student folders found (need names containing both 'layers#' and 'teacher4epoch').")
else:
    print("Evaluating NEW student folders:")
    for p in new_student_dirs: print("  ", p)

for path in new_student_dirs:
    try:
        student = load_local_model(path)
    except Exception as e:
        print("  Skipping (cannot load):", path, "=>", e)
        continue

    s_acc, s_f1 = evaluate_model(student, device)
    s_params = count_parameters(student) / 1e6
    s_layers = extract_layers(student)
    parsed_layers, parsed_seed = parse_meta_from_name(path)
    if parsed_layers is not None:
        s_layers = parsed_layers

    rel_acc = ((teacher_acc - s_acc) / teacher_acc * 100.0)
    rel_f1  = ((teacher_f1 - s_f1) / teacher_f1 * 100.0)
    param_red = (1.0 - (s_params / teacher_params)) * 100.0

    records.append({
        "Label": label_from_name(path),
        "Accuracy": s_acc,
        "Macro-F1": s_f1,
        "RelAccDrop%": round(rel_acc, 4),
        "RelF1Drop%": round(rel_f1, 4),
        "Layers": s_layers,
        "Params(M)": round(s_params, 2),
        "ParamReduction%": round(param_red, 2),
        "Path": path,
        "Seed": parsed_seed
    })

    student.to("cpu"); del student; gc.collect(); torch.cuda.empty_cache()

# ---------- Summarise ----------
df = pd.DataFrame(records)
if df.empty:
    print("No models evaluated.")
else:
    order = ["Teacher","Baseline","Soft","Hidden","Full","Embedding","Attention"]
    df["order"] = df["Label"].apply(lambda x: order.index(x) if x in order else len(order))
    df = df.sort_values(["order","Label"]).drop(columns="order").reset_index(drop=True)

    df.to_csv(OUT_CSV, index=False)
    try:
        df.to_excel(OUT_XLSX, index=False)
    except Exception as e:
        print("Excel save failed:", e)

    print("\nSaved results to:\n", OUT_CSV, "\n", OUT_XLSX)
    pd.set_option("display.max_colwidth", 160)
    display(df)

print("\nDone.")


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00
Device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

...

eval batches:   0%|          | 0/25 [00:00<?, ?it/s]

eval batches:   0%|          | 0/25 [00:00<?, ?it/s]


Saved results to:
 /content/drive/MyDrive/Colab Notebooks/CodeMix/distillation_results_rel_new_only.csv 
 /content/drive/MyDrive/Colab Notebooks/CodeMix/distillation_results_rel_new_only.xlsx


,Label,Accuracy,Macro-F1,RelAccDrop%,RelF1Drop%,Layers,Params(M),ParamReduction%,Path,Seed
0,Teacher,0.75250,0.752313,NaN,NaN,6,135.33,NaN,/content/drive/MyDrive/Colab Notebooks/CodeMix/results_teacher_4epoch/model,NaN
1,Baseline,0.73500,0.734196,2.3256,2.4081,2,106.97,20.95,/content/drive/MyDrive/Colab Notebooks/CodeMix/results_students/student_baseline_layers2_seed42_teacher4epoch,42.0
2,Soft,0.73125,0.730674,2.8239,2.8763,2,106.97,20.95,/content/drive/MyDrive/Colab Notebooks/CodeMix/results_students/student_soft_layers2_seed42_teacher4epoch,42.0
3,Hidden,0.68750,0.686275,8.6379,8.7780,2,106.97,20.95,/content/drive/MyDrive/Colab Notebooks/CodeMix/results_students/student_hidden_layers2_seed42_teacher4epoch,42.0
4,Full,0.72625,0.725302,3.4884,3.5904,2,106.97,20.95,/content/drive/MyDrive/Colab Notebooks/CodeMix/results_students/student_full_layers2_seed42_teacher4epoch,42.0
5,Embedding,0.73375,0.732905,2.4917,2.5798,2,106.97,20.95,/content/drive/MyDrive/Colab Notebooks/CodeMix/results_students/student_embedding_layers2_seed42_teacher4epoch,42.0
6,Attention,0.72750,0.726884,3.3223,3.3801,2,106.97,20.95,/content/drive/MyDrive/Colab Notebooks/CodeMix/results_students/student_attention_layers2_seed42_teacher4epoch,42.0



Done.
